In [124]:
# IMPORTACIONES
import time
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud
import geopandas as gpd
import pandas as pd
from datetime import datetime
from datetime import timedelta
from shapely import wkb

In [97]:
# CONECCION CON AMIGOCLOUD
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## PARA TRICHO

In [98]:
# PATHS
# RUTAS PARA NOTIFICACION DE LABORES DE LIBERACION DE TRICHOGRAMMA
path_xlsx_plan = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\pd_trichogramma_2024 - copia.xlsx'

# PILA DE MENSAJES
path_txt_msj = r'C:\Users\Usuario\Desktop\PYTHON\utea_scripts\notificaciones_whastapp\_msj_stack.txt'

## PARA PULVERIZACION

In [99]:
# PATHS
# RUTAS PARA NOTIFICACION DE LABORES DE LIBERACION DE TRICHOGRAMMA
path_xlsx_plan = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\CONTACTOS.xlsx'

# PILA DE MENSAJES
path_txt_msj = r'C:\Users\Usuario\Desktop\PYTHON\utea_scripts\notificaciones_whastapp\_msj_stack.txt'

In [100]:
# DATOS DE PILOTOS
tecnico_telf = {
    'ALEJANDRO SANCHEZ' : 78448317,
    'AURELIO GARCIA' : 79051559,
    'CHRISTIAN CALIZAYA' : 63619660,
    'EDUARDO REYES' : 75016609}

In [101]:
# EJECUTAR QUERY
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    # variable para almacenar resultado
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

In [102]:
# CORVERSION DE WKB A COORS
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

# NOTIFICACIONES DE LIBERACION

In [103]:
def get_registro_notificacion():
    # get data de nuevas notificacion de liberacion
    query = 'select * from dataset_354655 where procesado=false'
    notif = ejecutar_query_sql(33457, query, 'get')
    notif = notif['data']
    # extrae el primer elemento
    return notif

In [104]:
def get_planificacion():
    df_plan = pd.read_excel(path_xlsx_plan)
    df_plan = df_plan[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]
    return df_plan

In [105]:
def insertar_msj_stack(idd, fecha, hora, cod, nombre, telf, msj):
    linea = f'{idd};{fecha};{hora};{cod};{nombre};{telf};{msj}'
    with open(path_txt_msj, 'r') as file:
        contenido = file.readlines()
    contenido.insert(0, '\n')
    contenido.insert(0, linea)
    with open(path_txt_msj, 'w') as file:
        file.writelines(contenido)
    print(linea)

In [106]:
def anular_registro_notificacion(idd):
    query = f'update dataset_354655 set procesado = true where id = {idd}'
    exe = ejecutar_query_sql(33457, query, 'post')
    print('anulado:', idd)

In [107]:
# crea el mensaje 
def crear_msj(idd, fecha, hora, cod, nombre, telfs, labor, estado, tecnico, lon, lat):
    tec_telf = tecnico_telf[tecnico]
    for i in telfs:
        msj = ''
        if estado == 'INICIO DE LABOR':
            msj = f'''Señor(a) *{nombre}* se le informa que se esta iniciando el trabajo de *{labor}* en la siguiente ubicacion: \
https://www.google.com/maps?q={lat},{lon}. \
Para cualquier consulta puede comunicarse con ING. {tecnico} al {tec_telf}'''
        elif estado == 'FIN DE LABOR':
            msj = f'''Señor(a) *{nombre}* se le informa que se ha finalizado el trabajo de *{labor}* en la siguiente ubicacion: \
https://www.google.com/maps?q={lat},{lon}. \
Para cualquier consulta puede comunicarse con ING. {tecnico} al {tec_telf}'''
        insertar_msj_stack(idd, fecha, hora, cod, nombre, i, msj)

In [126]:
reg_amigo = get_registro_notificacion()

In [139]:
while True:
    reg_amigo = get_registro_notificacion()
    if len(reg_amigo) == 0:
        print('No existen notificaciones nuevas')
        continue
    reg_amigo = reg_amigo[0]
    # extrae cod_ca y nom_ca del registro
    cod_ca = int(reg_amigo['canhero'].split(' / ')[0])
    nom_ca = reg_amigo['canhero'].split(' / ')[1]
    # filtra en la lista de planificacion el cod_ca para obtener los numeros cel.
    reg = df_plan[df_plan['cod_ca']==cod_ca].iloc[0]
    # id del registro en amigocloud
    idd = reg_amigo['id']
    # extrae fecha y hora de registro menos 4 horas por desfase temprotal
    fecha_hora_obj = datetime.fromisoformat(reg_amigo['fecha_registro'])
    fecha_hora_menos_4h = fecha_hora_obj - timedelta(hours=4)
    # formatea la fecha de registro
    fecha_str = fecha_hora_menos_4h.strftime('%d-%m-%Y')
    # formatea la hora de registro
    hora_str = fecha_hora_menos_4h.strftime('%H:%M:%S')
    # extrae los numeros telefonicos
    telefonos = [reg['telf01'], reg['telf02'], reg['telf03']]
    # valida los numeros telefonicos
    telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
    # extrae nombre del piloto
    tecnico = reg_amigo['piloto']
    tec_telf = tecnico_telf[tecnico]
    telefonos_validos.append(tec_telf)
    telefonos_validos.append(68908131)
    # extra el tipo de labor (liberacion o pulverizacion)
    labor = reg_amigo['tipo_labor']
    # extrae el estado de la labor (inicio o fin de labor)
    estado = reg_amigo['tipo_mensaje']
    # extrae y convierte la ubicacion
    ubi = convertir_wkb(reg_amigo['ubicacion'])
    longitud = ubi.x  # Longitud (coordenada X)
    latitud = ubi.y   # Latitud (coordenada Y)
    crear_msj(idd, fecha_str, hora_str, cod_ca, nom_ca, telefonos_validos, labor, estado, tecnico, longitud, latitud)
    anular_registro_notificacion(idd)
    ##time.sleep(30)

10;27-09-2024;15:29:46;0;BISMARK SOCOMPI;78194371;Señor(a) *BISMARK SOCOMPI* se le informa que se esta iniciando el trabajo de *PULVERIZACION* en la siguiente ubicacion: https://www.google.com/maps?q=-17.1849563,-63.3144697. Para cualquier consulta puede comunicarse con ING. EDUARDO REYES al 75016609
10;27-09-2024;15:29:46;0;BISMARK SOCOMPI;62022996;Señor(a) *BISMARK SOCOMPI* se le informa que se esta iniciando el trabajo de *PULVERIZACION* en la siguiente ubicacion: https://www.google.com/maps?q=-17.1849563,-63.3144697. Para cualquier consulta puede comunicarse con ING. EDUARDO REYES al 75016609
10;27-09-2024;15:29:46;0;BISMARK SOCOMPI;75016609;Señor(a) *BISMARK SOCOMPI* se le informa que se esta iniciando el trabajo de *PULVERIZACION* en la siguiente ubicacion: https://www.google.com/maps?q=-17.1849563,-63.3144697. Para cualquier consulta puede comunicarse con ING. EDUARDO REYES al 75016609
10;27-09-2024;15:29:46;0;BISMARK SOCOMPI;68908131;Señor(a) *BISMARK SOCOMPI* se le informa que

KeyboardInterrupt: 

In [109]:
df_plan = pd.read_excel(path_xlsx_plan)

In [110]:
df_plan = df_plan[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]

In [111]:
df_plan.head(3)

,cod_ca,nom_ca,telf01,telf02,telf03
0,0,BISAMRK,78194371,62022996,NaN


In [112]:
# get data de nuevas notificacion de liberacion
query = 'select * from dataset_354655 where procesado=false'
notif = ejecutar_query_sql(33457, query, 'get')
notif = notif['data']

In [113]:
notif

[{'fecha_registro': '2024-09-27 18:55:02+00:00',
  'amigo_id': 'affa5abcf0444f2ab5e5b4fe706af68d',
  'id': 7,
  'piloto': 'ALEJANDRO SANCHEZ',
  'canhero': '0 / BISMARK SOCOMPI',
  'ubicacion': '0101000020E6100000B7E63345259B4FC020915B38B53A31C0',
  'tipo_labor': 'PULVERIZACION',
  'tipo_mensaje': 'INICIO DE LABOR',
  'procesado': False}]

In [114]:
# extrae el primer elemento
reg_amigo = notif[0]

In [117]:
reg_amigo

{'fecha_registro': '2024-09-27 18:55:02+00:00',
 'amigo_id': 'affa5abcf0444f2ab5e5b4fe706af68d',
 'id': 7,
 'piloto': 'ALEJANDRO SANCHEZ',
 'canhero': '0 / BISMARK SOCOMPI',
 'ubicacion': '0101000020E6100000B7E63345259B4FC020915B38B53A31C0',
 'tipo_labor': 'PULVERIZACION',
 'tipo_mensaje': 'INICIO DE LABOR',
 'procesado': False}

In [118]:
# extrae cod_ca y nom_ca del registro
cod_ca = int(reg_amigo['canhero'].split(' / ')[0])
nom_ca = reg_amigo['canhero'].split(' / ')[1]

In [119]:
# filtra en la lista de planificacion el cod_ca para obtener los numeros cel.
reg = df_plan[df_plan['cod_ca']==cod_ca].iloc[0]

In [120]:
# id del registro en amigocloud
idd = reg_amigo['id']

In [121]:
idd

7

In [30]:
# extrae fecha y hora de registro menos 4 horas por desfase temprotal
fecha_hora_obj = datetime.fromisoformat(reg_amigo['fecha_registro'])
fecha_hora_menos_4h = fecha_hora_obj - timedelta(hours=4)

In [31]:
# formatea la fecha de registro
fecha_str = fecha_hora_menos_4h.strftime('%d-%m-%Y')

In [32]:
# formatea la hora de registro
hora_str = fecha_hora_menos_4h.strftime('%H:%M:%S')

In [66]:
# extrae los numeros telefonicos
telefonos = [reg['telf01'], reg['telf02'], reg['telf03']]
# valida los numeros telefonicos
telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]

In [67]:
# extra el tipo de labor (liberacion o pulverizacion)
labor = reg_amigo['tipo_labor']

In [68]:
# extrae el estado de la labor (inicio o fin de labor)
estado = reg_amigo['tipo_mensaje']

In [69]:
# extrae nombre del piloto
tecnico = reg_amigo['piloto']

In [70]:
# extrae y convierte la ubicacion
ubi = convertir_wkb(reg_amigo['ubicacion'])
longitud = ubi.x  # Longitud (coordenada X)
latitud = ubi.y   # Latitud (coordenada Y)

In [74]:
crear_msj(idd, fecha_str, hora_str, cod_ca, nom_ca, telefonos_validos, labor, estado, tecnico, longitud, latitud)

5;27-09-2024;14:16:07;0;BISMARK SOCOMPI;78194371;Señor(a) *BISMARK SOCOMPI* se le informa que se esta iniciando el trabajo de *PULVERIZACION* en la siguiente ubicacion: https://www.google.com/maps?q=-17.2435023,-63.1980813. Para cualquier consulta puede comunicarse con ING. ALEJANDRO SANCHEZ al 78448317
5;27-09-2024;14:16:07;0;BISMARK SOCOMPI;62022996;Señor(a) *BISMARK SOCOMPI* se le informa que se esta iniciando el trabajo de *PULVERIZACION* en la siguiente ubicacion: https://www.google.com/maps?q=-17.2435023,-63.1980813. Para cualquier consulta puede comunicarse con ING. ALEJANDRO SANCHEZ al 78448317


In [44]:
telefonos_validos

[]